Your name here.  
Your section number here.  

# Workshop 11: Root Finding and Interpolation Methods

## Due by 08:59 pm Tuesday, November 7, 2023


**Submit this notebook to bCourses to receive a grade for this Workshop.**

Please complete workshop activities in code cells in this iPython notebook. The activities titled **Practice** are purely for you to explore Python, and no particular output is expected. Some of them have some code written, and you should try to modify it in different ways to understand how it works. Although no particular output is expected at submission time, it is _highly_ recommended that you read and work through the practice activities before or alongside the exercises. However, the activities titled **Exercise** have specific tasks and specific outputs expected. Include comments in your code when necessary. Enter your name in the cell at the top of the notebook. 

**The workshop should be submitted on bCourses under the Assignments tab (both the .ipynb and .pdf files).**

In lecture, we discussed a few different ways to estimate roots of nonlinear functions of one variable. Here we will expand on the details and use some of those techniques.

# Part 1 Root finding

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Root finding using the bisection method

First we introduce the `bisect` algorithm which is (i) robust and (ii) slow but conceptually very simple.

Suppose we need to compute the roots of 

$$f(x) = x^3 - 2x^2$$

This function has a roots at $x=0, 2$. Run the cell below to generate a plot of $f(x)$. What do you notice about how the function behaves around each of these two zeros?

In [ ]:
def f(x):
    return x ** 3 - 2 * x ** 2

# Visualize f(x) and see the roots
xd = np.linspace(-1,3,100)
yd = f(xd)
plt.figure()
plt.plot(xd,yd, label='f(x)')
plt.plot(xd,np.zeros(len(xd)))
plt.legend()
plt.show()

Indeed, the `bisect` method operates on the "Intermediate Value Theorem" which just makes the observation that if a continuous function $f(x)$ changes sign over an interval $x\in [a,b]$, then there must exist at least one value of $x$ in that interval for which $f(x) = 0$. As a result, this method cannot find the root at $x=0$. So to use `scipy.optimize.bisect` you need to give it three arguments: the name of the python function which encodes $f(x)$, the left end of your interval ($a$) and the right end of your interval $b$. But further more, $a$ and $b$ must be such that $f(a)$ and $f(b)$ have opposite sign. Try changing some of these values in the call to `bisect` below!

In [ ]:
from scipy.optimize import bisect


x_root = bisect(f, 1.5, 3, xtol=1e-6) #xtol is an optional argument specifying how precise we want the answer
# Note that the input arguments are the following
# 1. f the function for which you want to find the roots
# 2. "1.5" the lower value (xl) that you use to bracket the root. You should adjust it based on the function 
# you have to solve
# 3. "3" is the upper value (xu) that you use to bracket the root.  You should adjust it based on the function 
# you have to solve
# 4. xtol = 1e-6 is the tolerance. The smaller, the more precise the roots are


print("The root x is approximately x=%14.12g,\n"
      "the error is less than 1e-6." % (x_root))
print("The exact error is %g." % (2 - x_root))

# Root finding using Brent method

This is another method to find a root of the function $f(x)$ on the sign changing interval $x\in[a , b]$. It is a safe version of the secant method that uses inverse quadratic extrapolation. Brent’s method combines a few other elementary root-finding techniques: root bracketing, interval bisection, and inverse quadratic interpolation. But again, this still requires that $f(a)$ and $f(b)$ have opposite signs.

In [ ]:
from scipy.optimize import brentq

def f(x):
    return x ** 3 - 2 * x ** 2

x = brentq(f, 0.5,3, xtol=1e-6)

print("The root x is approximately x=%14.12g,\n"
      "the error is less than 1e-6." % (x))
print("The exact error is %g." % (2 - x))

# Exercise 1
Use the built-in functions (`bisect` and/or `brentq`) to compute the roots of the following functions: 
* $f(x) = e^{x} - x - 2$
* $f(x) = \cos(\frac{\pi x}{2})-x$

**Here the key is to figure out xl, xu values that bracket the roots**
* you will see for some functions it is not so trivial to identify proper values of xl and xu

In [ ]:
# Code for Exercise 1

# Root finding using the `fsolve` function

A (often) better (in the sense of “more efficient”) algorithm than the bisection algorithm is implemented in the general purpose `fsolve()` function for root finding of (multidimensional) functions. This algorithm needs only one starting point close to the suspected location of the root (but is not garanteed to converge).

Here is an example:

In [ ]:
from scipy.optimize import fsolve

def f(x,a,b):
    return a*x**3 - b*x**2

a = 1
b = 2
x = fsolve(f, x0=[-0.5,4], args=(a,b))           # look for two roots starting with 0 and 3
# here the input arguments are
# 1. "f" the function for which you want to find a solution
# 2. "x0" the initial guess of solutions
# the length of x0 must be at least as large as the number of roots
# in this example, there are two roots, 0 and 2
# so x0 = [-0.5,4]
# however, it would also work if you have x0 = [-0.5,4,2, 4]
# try it yourself and see what happens in that case
# args=(a,b) are parameters in the function "f" that you can specify



print("Number of roots is", len(x))
print("The root(s) are ", x)

The input to `fsolve` is the name of the python function and the array of initial locations for the roots you are trying to find. You can optionally pass additional arguments (parameters) to the function as a list with the keyword `args`. The return value of `fsolve` is a numpy array of the best estimates of the locations of the roots found for each initial guess given. If $n$ initial guesses are given, $n$ estimates are returned.

## Exercise 2

Find the solutions of the quadratic equation $ax^2 + bx + c=0$ for an arbitrary set of coefficients $a$, $b$, $c$ using `fslove` , and compare to the exact solution.

* Write a single function that takes values of a,b,c as the input arguments
* the function prints out the solutions to $ax^2 + bx + c=0$, for the given a,b,c values
* Use this function to compute roots for the following scenarios
    * a,b,c = 1,2,1
    * a,b,c = 1,3,5
    * a,b,c = 12.5, -6.4, -1.25
    * a,b,c = 4,2,1

In [ ]:
# Code for Exercise 2



# Part 2 Interpolation Techniques

In this exercise, we will explore three different interpolation techniques to approximate a function from a set of sampled data points. The function under consideration is:

$\large f(x) = \sin(x) \times e^{-x} \times \log(x) $

We will sample this function over the range [1, 10] at ten distinct points. Using this sampled data, we will implement the following interpolation techniques:

1. **Global Lagrange Interpolation**: This method constructs a polynomial that passes through each data point.
2. **Cubic Spline Interpolation**: A piecewise cubic polynomial is fitted between every two consecutive data points ensuring smoothness.
3. **Hermite Cubic Spline Interpolation**: This method is an enhancement of cubic spline interpolation. It takes into account not only the function values but also their derivatives. We will estimate the derivatives using the central difference method directly on the original function.

For each technique, we will plot the interpolated function alongside the original function to visually inspect the accuracy and characteristics of the interpolation.

Let's dive into the code and visualize the results!


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the function we want to sample.
def original_function(x):
    return np.sin(x) * np.exp(-x) * np.log(x)

# Define the range and sample the function over 10 points between 1 and 10.
x_sample = np.linspace(1, 10, 10)
y_sample = original_function(x_sample)

# Define a dense range to plot the original function smoothly.
x_dense = np.linspace(1, 10, 400)
y_dense = original_function(x_dense)

# Plot the original function.
plt.plot(x_dense, y_dense, label='Original Function', color='blue')

# Plot the sampled points.
plt.scatter(x_sample, y_sample, color='red', label='Sampled Points')
plt.title('Sampled Data & Original Function')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()


### Global Lagrange Interpolation

Global Lagrange Interpolation is a method used to approximate a function based on a given set of data points. It constructs a polynomial that passes through each of these data points. Specifically, for \(n+1\) data points, a polynomial of degree \(n\) is formed.

**Degrees of Freedom (DOFs) and Constraints:**

For the Lagrange interpolation, the number of Degrees of Freedom (DOFs) corresponds to the coefficients of the polynomial. If there are \(n+1\) data points, the polynomial will have \(n+1\) coefficients, hence \(n+1\) DOFs.

The constraints in this method are the given data points themselves. Each data point provides a constraint that the interpolating polynomial must satisfy. Therefore, for \(n+1\) data points, there are \(n+1\) constraints.

In summary, the number of DOFs and constraints are equal in Global Lagrange Interpolation, ensuring a unique polynomial that passes through all the provided data points.


In [ ]:
from scipy.interpolate import lagrange

# Construct the Lagrange interpolating polynomial using the sampled data.
lagrange_poly = lagrange(x_sample, y_sample)

# Evaluate the Lagrange polynomial for the dense x-values.
y_lagrange = lagrange_poly(x_dense)

# Plot the original function.
plt.plot(x_dense, y_dense, label='Original Function', color='blue')

# Plot the result of the Lagrange interpolation.
plt.plot(x_dense, y_lagrange, label='Lagrange Interpolation', linestyle='--', color='green')

# Highlight the sampled points.
plt.scatter(x_sample, y_sample, color='red', label='Sampled Points')
plt.title('Lagrange Interpolation & Original Function')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()


### Cubic Spline Interpolation

Cubic Spline Interpolation is a piecewise interpolation method where each segment between two consecutive data points is approximated by a cubic polynomial. The special feature of these splines is that they are twice continuously differentiable, ensuring smooth transitions between the segments.

**Degrees of Freedom (DOFs) and Constraints:**

1. **Degrees of Freedom (DOFs):** For \(n+1\) data points, there will be \(n\) intervals, and each interval is represented by a cubic polynomial, which has 4 coefficients. Hence, there would theoretically be \(4n\) DOFs.

2. **Constraints:** 
   - **Function Values:** The polynomial must pass through each of the \(n+1\) data points, providing \(2n\) constraints (2 for each interval, one at the start and one at the end).
   - **First Derivative Continuity:** The first derivatives of the polynomials at the interior points (except endpoints) must be equal, adding \(n-1\) constraints.
   - **Second Derivative Continuity:** The second derivatives of the polynomials at the interior points (except endpoints) must be equal, adding another \(n-1\) constraints.
   
   Together, this provides a total of \(4n-2\) constraints. 

Two more constraints are often introduced by setting the second derivative to zero at the endpoints (natural spline), or by specifying the first derivatives at the endpoints (clamped or complete spline), making the total constraints equal to the DOFs.

In summary, Cubic Spline Interpolation constructs a set of cubic polynomials that not only pass through all the given data points but also ensure smooth transitions and continuous first and second derivatives across segments.


In [ ]:
from scipy.interpolate import CubicSpline

# Construct the cubic spline interpolation using the sampled data.
cubic_spline = CubicSpline(x_sample, y_sample)

# Evaluate the cubic spline for the dense x-values.
y_cubic = cubic_spline(x_dense)

# Plot the original function.
plt.plot(x_dense, y_dense, label='Original Function', color='blue')

# Plot the result of the cubic spline interpolation.
plt.plot(x_dense, y_cubic, label='Cubic Spline Interpolation', linestyle='--', color='green')

# Highlight the sampled points.
plt.scatter(x_sample, y_sample, color='red', label='Sampled Points')
plt.title('Cubic Spline Interpolation & Original Function')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()


### Hermite Cubic Spline Interpolation

Hermite Cubic Spline Interpolation is an enhanced interpolation method that uses both function values and derivative values to approximate a function. While it is similar to the regular cubic spline interpolation, Hermite interpolation constructs the cubic polynomial based on the function values at the data points as well as their first derivatives.

**Degrees of Freedom (DOFs) and Constraints:**

1. **Degrees of Freedom (DOFs):** For \(n+1\) data points, there will be \(n\) intervals, and each interval is represented by a cubic polynomial, which has 4 coefficients. Thus, there would theoretically be \(4n\) DOFs.

2. **Constraints:** 
   - **Function Values:** The polynomial must pass through each of the \(n+1\) data points. This provides \(2n\) constraints (2 for each interval: one at the start and one at the end).
   - **Derivative Values:** The polynomial's first derivative at the data points must match the provided or computed first derivative values. This gives another \(2n\) constraints.
   
   Together, this results in \(4n\) constraints.

Given that the number of DOFs and constraints are equal in Hermite Cubic Spline Interpolation, a unique cubic polynomial is determined for each interval that satisfies both the function values and the derivative values at the data points. This ensures that the interpolating function not only passes through the data points but also aligns with the specified slope or trend of the function at those points.


#### Central Difference Method

In our exercises, we use the central difference method to compute the derivative values required for Hermite Cubic Spline Interpolation from the original function directly.


The Central Difference Method is a numerical technique used to approximate the derivative of a function. It's based on the idea of symmetrically approximating the slope of the function using two nearby points. The method provides a better approximation to the derivative than the forward or backward difference methods, especially when the spacing between points is small.

##### Formula:

Given a function \( f(x) \), the central difference approximation of its derivative \( f'(x) \) at a point \( x \) using a small step size \( h \) is given by:

$ \large f'(x) \approx \frac{f(x+h) - f(x-h)}{2h} $

##### Usage:

The central difference method is particularly useful when analytical differentiation is difficult or when working with discrete data sets where the function form might not be known. However, the accuracy of the approximation depends on the choice of \( h \). While a smaller \( h \) generally yields a more accurate estimate, it may also introduce numerical instability or rounding errors, especially for very small values of \( h \).





In [ ]:
from scipy.interpolate import CubicHermiteSpline

# Define the function we want to sample.
def original_function(x):
    return np.sin(x) * np.exp(-x) * np.log(x)

def central_difference(func, x, h=1e-5):
    """
    Compute the derivative of the function using the central difference method.
    :param func: The function to differentiate.
    :param x: Points at which the derivative should be estimated.
    :param h: Small difference for differentiation.
    :return: Estimated derivative values at the points x.
    """
    return (func(x + h) - func(x - h)) / (2 * h)

# Using the central_difference method with the original function to compute derivatives at the sampled points.
dy_sample = central_difference(original_function, x_sample)

# Construct the Hermite cubic spline using the sampled data and computed derivatives.
hermite_spline = CubicHermiteSpline(x_sample, y_sample, dy_sample)

# Evaluate the Hermite spline for the dense x-values.
y_hermite = hermite_spline(x_dense)

# Plot the original function.
plt.plot(x_dense, y_dense, label='Original Function', color='blue')

# Plot the result of the Hermite cubic spline interpolation.
plt.plot(x_dense, y_hermite, label='Hermite Cubic Spline Interpolation', linestyle='--', color='green')

# Highlight the sampled points.
plt.scatter(x_sample, y_sample, color='red', label='Sampled Points')
plt.title('Hermite Cubic Spline Interpolation & Original Function')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()


### Exercise: 

In this exercise, you will explore the behavior of the function:

$\large y = \frac{1}{{\sqrt{1-x^2}}} $

You'll sample this function at a few specific points and then use two different interpolation methods to estimate its behavior. You will then compare the results of these interpolations to the original function.

#### Tasks

1. Define the function $y = \frac{1}{{\sqrt{1-x^2}}}$.
2. Sample the function at the following $x$ values: 
    - $0.0001, 0.0002, 0.0003$, $0.92, 0.95, 0.99$
3. Perform a global Lagrange interpolation on these sampled points.
4. Perform a cubic spline interpolation on these sampled points.
5. Plot:
    - The original function
    - The Lagrange interpolation
    - The cubic spline interpolation
    - The sampled points
6. Discuss the results. In particular:
    - How do the two interpolation methods compare to each other?
    - How well do they approximate the original function between the sampled points?
    - Do you notice any peculiar behavior outside the sampled region? If so, can you explain it?



In [ ]:
# Code for the exercise